# Monte Carlo Markov Chain Method
* Oscar A. Esquivel-Flores
* Óscar A. Alvarado Morán

Vamos a utilizar el paquete `LinearAlgebra` para calcular normas y obtener matrices especiales, entre otras cosas.

In [13]:
using LinearAlgebra, BenchmarkTools

Sea el sistema de ecuaciones lineales algebraico:

\begin{align}
3x -2y & = & -3 \\ 
x + 3 - y & = & -3 \\ 
2x + y + 4z & = & 6 
\end{align}

En la representación matricial del sistema $Ax = b$, la matriz $A$ corresponde a los coeficientes, el vector $x$ contiene las variables del sistema y el vector $b$ corresponde a los términos independientes.


In [17]:
A = [3 -2 0; 1 3 -1; 2 1 4]
b = [-3, -3, 6]

3-element Array{Int64,1}:
 -3
 -3
  6

In [18]:
A

3×3 Array{Int64,2}:
 3  -2   0
 1   3  -1
 2   1   4

In [19]:
b

3-element Array{Int64,1}:
 -3
 -3
  6

Usamos el operador `\` de Juia(base) para calcular la solución del sistema, dados $A$ y $b$.

In [20]:
A\b

3-element Array{Float64,1}:
 -1.0
  0.0
  2.0

La solución del sistema es $x = [-1, 0 ,2]$

El método de Monte Carlo-Cadenas de Markov inicia descomponiendo la matriz $A = M - N$ para obtener la expresión iterativa:

\begin{equation}
x_{k+1} = Tx_{k} + f,
\end{equation}

para $k = 0, 1, 2, ...,$

donde $T=M^{-1}N$ y $f = M^{-1}b$

### Contrucción de M y N

En Julia la matriz identidad se construye mediante una sintaxis un poco freaky, la ilustro abajo, la vamos a utilizar más adelante...

In [21]:
Matrix{Float64}(I, 3, 3)

3×3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

Una forma de construir $M$ es extraer la diagonal principal de $A$ y rellenar de ceros,

In [22]:
M = Diagonal(A) # Tal vez esta es una mejor forma(?)

3×3 Diagonal{Int64,Array{Int64,1}}:
 3  ⋅  ⋅
 ⋅  3  ⋅
 ⋅  ⋅  4

$N$ la obtenemos sustituyendo $M$ de $A = M-N$

In [23]:
N = M - A

3×3 Array{Int64,2}:
  0   2  0
 -1   0  1
 -2  -1  0

Obtenemos la matriz $T$ haciendo $M^{-1}N$

In [24]:
T = inv(M) * N

3×3 Array{Float64,2}:
  0.0        0.666667  0.0
 -0.333333   0.0       0.333333
 -0.5       -0.25      0.0

La norma de $T$ y el valor de sus eigenvalores nos permite conocer la convergencia del método. La norma-`p`está definida como:

$$ \|A\|_p = \left( \sum_{i=1}^n | a_i | ^p \right)^{1/p} $$

donde `p` es cualquier número entero.

La función `norm()` de Julia calcula la norma 2 de un vector...

In [25]:
norm(T)

0.9895285072531598

In [26]:
eigvals(T)

3-element Array{Complex{Float64},1}:
 -0.2865958167935963 + 0.0im
 0.14329790839679796 - 0.6059359926660953im
 0.14329790839679796 + 0.6059359926660953im

Si los eigenvalores de $T$ están dentro del círculo unitario o la norma 2 de $T$ es menor que uno, se garantiza la convergencia del método. 

Obtenemos el vector $f$ haciendo $M^{-1}b$

In [27]:
f = inv(M) * b

3-element Array{Float64,1}:
 -1.0
 -1.0
  1.5

In [28]:
nT, mT = size(T)

(3, 3)

El proceso de Markov nos va a permitir estimar cada elemento del vector solución $x$, el proceso consiste en crear una cadena de estados que pueden interpretarse como una secuencia de elecciones aleatorias de elementos de $T$ y $f$ con los cuales crear pesos (cocientes) de la forma:

\begin{equation}
W_m = W_{m-1}\cdot w_{r_{m-1}r_m}
\end{equation}
donde $w_{r_{m-1}r_m}=\dfrac{ t_{r_{m-1}r_m}} {p_{r_{m-1}r_m} }$ y $W_0 = \dfrac{h_{r_0}}{p_{r_0}}$

Para hacer la secuencia aleatoria se requiere construir una matriz de probabilidad $P$ para elegir cada elemento no cero de $T$. Iniciamos creando un vector $S$ en donde se guardan el número de elementos no cero de $T$ por cada renglón. 

Definimos la matriz $S$ donde se guardarán los valores no cero por renglón. Este vector se usa para calcular la matriz de probabilidad $P$. Usamos una _comprehension list_ de arreglos para recorrer $T$.

In [29]:
S = zeros(Int64, nT)

3-element Array{Int64,1}:
 0
 0
 0

In [30]:
[S[i] += 1 for i in 1:nT, j in 1:mT if T[i,j] != 0]

5-element Array{Int64,1}:
 1
 1
 1
 2
 2

In [31]:
S

3-element Array{Int64,1}:
 1
 2
 2

Cada renglón de la matriz $P$ contiene valores de probabilidad (uniforme) para cada elemento de $T$ diferentes de cero.

In [32]:
P = zeros(nT, mT)
[P[i,j] = 1/S[i] for i in 1:nT, j in 1:mT if T[i,j] != 0];

In [33]:
P

3×3 Array{Float64,2}:
 0.0  1.0  0.0
 0.5  0.0  0.5
 0.5  0.5  0.0

El vector $P_i$ contiene la probabilidad uniforme de elegir un elemento no cero de $f$

In [34]:
Pi = [1/nT for i in 1:nT]

3-element Array{Float64,1}:
 0.3333333333333333
 0.3333333333333333
 0.3333333333333333

Dos parámetros nos van a permitir obtener cierta precisión en la estimación. Definimos el error de exactitud $\epsilon$ y el error estadístico $\delta$. 

In [8]:
ϵ = 0.5
δ = 0.5

0.5

Para estimar cada elemento del vector solución se requiere construir $N$ cademas de Markov y obtener el promedio. El número de cadenas de Markov $N$ se calcula haciendo:

In [9]:
N = floor((0.6745/δ)^2*((norm(f)^2)/(1-norm(T))^2)) + 1

70534.0

Para mayor exactitud en la estimación los valores de los errores $\epsilon$ y $\delta$ deberán reducirse, lo que se traduce en cadenas más grandes, resultando en el incremento del valor de $N$ y por tanto los cálculos.

El algoritmo Monte Carlo-Cadena de Markov se implementa a continuación:

In [23]:
function MCMC(N,T,f,P,ϵ)
    nT, mT = size(T)
    X = zeros(Float32, mT)
    cont1 = 0
    cont2 = 0
    for column_T in 1:nT # Separemos todo primero por columnas. Por qué por columnas(mT)?
        W_0 = 1
        R = 0 
        for chain in 1:N # Número de Cadenas de Markov
            W = W_0
            point = column_T # ---------------------------------------------------Aquí
            X[column_T] = W_0 * f[column_T] # ---------------------------------------Aquí
            while abs(W) >= ϵ
                nextpoint  = 1
                u = rand()
                while u >= sum(P[point, 1:nextpoint])
                    nextpoint += 1
                    cont1 += 1
                end
                if T[point, nextpoint] != 0 # Qué pasa cuando no se aplica esto?
                    W_new = W *(T[point, nextpoint]/P[point, nextpoint])
                    X[column_T] += W_new * f[nextpoint] # ------------------------Aquí
                    cont2 += 1
                end
                W = W_new
                point = nextpoint
            end
            R += X[column_T] # ---------------------------------------------------Aquí
        end
        println(R/N)
    end
    @show cont1, cont2
end

MCMC (generic function with 1 method)

In [53]:
MCMC(7.053354e6,T,f,P,ϵ)

InterruptException: InterruptException:

In [24]:
MCMC(70534,T,f,P,ϵ)

-1.1109822
0.41707262
2.0966182
(cont1, cont2) = (423514, 476082)


(423514, 476082)

Tabla comparativa:

| $\epsilon$  | $\delta$ | no. cadenas |    Aproximación $x$    | tiempo  |
| -------- | ----------- |-------------| -----------------------| ------- |
| 0.5      | 0.5         |   70534     | [-1.1114, 0.416, 2.09] |    4.024 s    |
| 0.05     | 0.05        | 7.053354e6  | [-0.975 , 0.003, 1.978] |    412.273 s    |


Algunas preguntas surgen:
1. El algoritmo funciona para matrices que no tiene $T$ diagonal dominante, es decir que no tengan norma 2 menor que uno o los eigenvalores no estén dentro del círculo unitario?
2. Si las dimensiones del sistema aumentan, ¿sigue siendo efectivo el método?